In [ ]:
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, select, func, desc
from gorillatracker.ssl_pipeline.models import Video, VideoFeature

from gorillatracker.ssl_pipeline.dataset import GorillaDataset
from gorillatracker.ssl_pipeline.queries import load_processed_videos


In [ ]:
dataset = GorillaDataset("sqlite:///../test.db")

In [ ]:
with Session(dataset.engine) as session:
    videos = load_processed_videos(session, "2024-04-09", [])
    test = session.execute(
        select(Video.fps, func.sum(Video.frames).label("count"), Video.path)
        .group_by(Video.video_id)
        .order_by(desc("count"))
        )
    
#print(videos)
print(test.all())

In [ ]:
with Session(dataset.engine) as session:
    video_id = session.execute(select(Video.video_id).where(Video.path.endswith("R033_20220403_392.mp4")))
    
print(video_id.scalar())

In [ ]:
with Session(dataset.engine) as session:
    session.add(VideoFeature(video_id=1, type="test", value="testvalue"))
    session.commit()

In [ ]:
with Session(dataset.engine) as session:
    get_video_features = (select(VideoFeature.video_id, VideoFeature.feature_id, VideoFeature.value, VideoFeature.type)
                          .group_by(VideoFeature.feature_id))
    print(session.execute(get_video_features).all())

In [ ]:
from sqlalchemy import create_engine, MetaData

# Create a metadata object
metadata = MetaData()

# Reflect the tables from the database
metadata.reflect(bind=dataset.engine)

# Print all table names
print("Tables in the database:")
for table in metadata.tables.values():
    print(table.name)